In [3]:
%run custom_tools.py

current_conda_env = !echo $CONDA_DEFAULT_ENV
check_conda_env(current_conda_env[0])

Setup environment... done!


✅ <span style='color:green'>Working on **impact-annotator_env** conda environment.</span>

In [15]:
impact = pd.read_csv('../../data/final_IMPACT_mutations_180508.txt', sep = '\t', low_memory = False)

In [16]:
impact = impact.iloc[0:10000]

In [17]:
impact.shape

(10000, 58)